# 💾 Notebook 5: DVC Data Versioning

**Author:** Amey Talkatkar | **Course:** MLOps with Agentic AI

## 🎯 Learning Objectives
- Initialize DVC in your project
- Track datasets with DVC
- Push/pull data from remote storage (DagsHub)
- Switch between data versions
- Create DVC pipelines
- Integrate DVC with Git

## 🔥 The Problem
You have:
- `sales_data_v1.csv` (January data)
- `sales_data_v2.csv` (added February)
- `sales_data_final.csv` (added March)
- `sales_data_final_v2.csv` (fixed bugs)
- `sales_data_actually_final.csv` (really final!)

Which version trained the production model? **Nobody knows!** 😱

**DVC Solution:** One file name, infinite versions tracked in Git!

## Step 1: Setup

⚠️ **Note:** This notebook demonstrates DVC commands. Most commands need to be run in terminal, not in notebook cells.

In [ ]:
import os
import subprocess
import pandas as pd

# Change to project root
os.chdir('..')
print(f"Current directory: {os.getcwd()}")

## Step 2: Initialize DVC

Run in terminal:
```bash
cd ~/sales-forecasting-mlops
dvc init
```

This creates:
- `.dvc/` directory (DVC configuration)
- `.dvcignore` file (like .gitignore)

In [ ]:
# Check if DVC is initialized
if os.path.exists('.dvc'):
    print("✅ DVC already initialized")
else:
    print("❌ DVC not initialized. Run: dvc init")

## Step 3: Add Data to DVC

Track your dataset with DVC:

```bash
dvc add data/raw/sales_data.csv
```

This creates:
- `data/raw/sales_data.csv.dvc` (metadata file - track in Git)
- Updates `data/raw/.gitignore` (ignore actual data)
- Moves data to `.dvc/cache/`

In [ ]:
# Simulate dvc add
data_file = 'data/raw/sales_data.csv'
dvc_file = f'{data_file}.dvc'

if os.path.exists(dvc_file):
    print(f"✅ {dvc_file} exists")
    
    # Read .dvc file
    with open(dvc_file, 'r') as f:
        print("\n📄 Contents of .dvc file:")
        print(f.read())
else:
    print(f"❌ Run: dvc add {data_file}")

## Step 4: Commit to Git

Add .dvc file to Git (NOT the data!):

```bash
git add data/raw/sales_data.csv.dvc data/raw/.gitignore
git commit -m "Add sales data v1 with DVC"
```

Now Git tracks:
- ✅ `.dvc` file (123 bytes)
- ❌ `.csv` file (2 MB) - ignored!

## Step 5: Configure DVC Remote (DagsHub)

Set up remote storage:

```bash
# Add DagsHub as remote
dvc remote add origin https://dagshub.com/YOUR_USERNAME/sales-forecasting-mlops.dvc

# Set credentials
dvc remote modify origin --local auth basic
dvc remote modify origin --local user YOUR_USERNAME
dvc remote modify origin --local password YOUR_TOKEN

# Set as default
dvc remote default origin
```

In [ ]:
# Check DVC configuration
result = subprocess.run(['dvc', 'remote', 'list'], capture_output=True, text=True)
print("DVC Remotes:")
print(result.stdout if result.returncode == 0 else "❌ No remotes configured")

## Step 6: Push Data to Remote

Upload data to DagsHub:

```bash
dvc push
```

This uploads actual data files to DagsHub storage.

## Step 7: Simulate Data Update

Let's create a new version of our data.

In [ ]:
# Generate larger dataset (v2)
print("📊 Generating updated dataset...")
result = subprocess.run(
    ['python', 'data/generate_data.py', '--rows', '20000', '--output', 'data/raw/sales_data.csv'],
    capture_output=True,
    text=True
)

if result.returncode == 0:
    print("✅ New data generated (20,000 rows)")
    
    # Check file size
    size_mb = os.path.getsize('data/raw/sales_data.csv') / 1024 / 1024
    print(f"   File size: {size_mb:.2f} MB")
else:
    print(f"❌ Error: {result.stderr}")

Now track the new version:

```bash
# Track new version
dvc add data/raw/sales_data.csv

# Notice the .dvc file changed (different MD5)
git diff data/raw/sales_data.csv.dvc

# Commit the change
git add data/raw/sales_data.csv.dvc
git commit -m "Update sales data v2 - 20,000 rows"

# Push to DVC remote
dvc push
```

## Step 8: Switch Between Versions

The magic of DVC - time travel for data!

```bash
# See Git history
git log --oneline data/raw/sales_data.csv.dvc

# Output:
# abc123 Update sales data v2 - 20,000 rows
# def456 Add sales data v1 - 10,000 rows

# Switch to v1
git checkout def456 data/raw/sales_data.csv.dvc
dvc checkout

# Now you have v1 data!
wc -l data/raw/sales_data.csv
# Output: 10001 (10,000 rows + header)

# Switch back to v2
git checkout main data/raw/sales_data.csv.dvc
dvc checkout

# Now you have v2 data!
wc -l data/raw/sales_data.csv
# Output: 20001 (20,000 rows + header)
```

## Step 9: DVC Pipelines (Advanced)

Track data processing pipelines:

```bash
# Define a preprocessing stage
dvc run -n preprocess \
    -d data/raw/sales_data.csv \
    -o data/processed/features.csv \
    python src/data/data_processor.py
```

This creates `dvc.yaml`:
```yaml
stages:
  preprocess:
    cmd: python src/data/data_processor.py
    deps:
      - data/raw/sales_data.csv
    outs:
      - data/processed/features.csv
```

In [ ]:
# Check if dvc.yaml exists
if os.path.exists('dvc.yaml'):
    print("✅ DVC pipeline exists")
    with open('dvc.yaml', 'r') as f:
        print("\n📄 Pipeline:")
        print(f.read())
else:
    print("ℹ️ No DVC pipeline yet (optional)")

## Step 10: DVC in Airflow

How to use DVC in production pipelines:

```python
# In Airflow DAG
from airflow.operators.bash import BashOperator

pull_data = BashOperator(
    task_id='dvc_pull',
    bash_command='cd /path/to/project && dvc pull',
    dag=dag
)

train_model = PythonOperator(
    task_id='train',
    python_callable=train_function,
    dag=dag
)

pull_data >> train_model
```

## 📊 Real-World Example

In [ ]:
# Demonstrate versioning benefits
print("📈 Data Versioning Benefits:\n")

versions = [
    {"date": "2024-01-01", "rows": 10000, "model_rmse": 1250, "commit": "abc123"},
    {"date": "2024-02-01", "rows": 15000, "model_rmse": 1100, "commit": "def456"},
    {"date": "2024-03-01", "rows": 20000, "model_rmse": 980, "commit": "ghi789"},
]

df_versions = pd.DataFrame(versions)
display(df_versions)

print("\n💡 With DVC:")
print("  1. Can reproduce any model (just checkout the commit)")
print("  2. Know exactly which data trained which model")
print("  3. Team members get same data with 'dvc pull'")
print("  4. No 'sales_data_final_v2_actually_final.csv' nonsense!")

## ✅ DVC Workflow Summary

### Daily Workflow:
```bash
# 1. Get latest code and data
git pull
dvc pull

# 2. Make changes to data
python data/generate_data.py --rows 50000

# 3. Track with DVC
dvc add data/raw/sales_data.csv

# 4. Commit to Git
git add data/raw/sales_data.csv.dvc
git commit -m "Update data to 50k rows"

# 5. Push everything
git push
dvc push
```

### Why This Matters for MLOps:
- 🔄 **Reproducibility**: Exact data version for each model
- 💾 **Storage Efficient**: Git stores only metadata
- 👥 **Team Collaboration**: Everyone gets same data
- 📊 **Data Lineage**: Track data evolution
- 🚀 **CI/CD Ready**: Automatic data pulls in pipelines

### Common Commands:
```bash
dvc add <file>              # Track file
dvc push                    # Upload to remote
dvc pull                    # Download from remote
dvc checkout                # Get data for current commit
dvc status                  # Check if data changed
dvc repro                   # Run pipeline
```

---

**Next:** `06_Airflow_Pipeline_Development.ipynb` - Automate everything!

**© 2024 Amey Talkatkar**